In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
import json
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation, BatchNormalization
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils 

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fer2013.csv")

In [5]:
df.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
df.groupby('Usage')['Usage'].agg('count')

Usage
PrivateTest     3589
PublicTest      3589
Training       28709
Name: Usage, dtype: int64

In [6]:
X_train, y_train, X_test, y_test = [], [], [], []

In [7]:
for index,row in df.iterrows():
  val=row['pixels'].split(" ")
  try:
    if 'Training' in row['Usage']:
      X_train.append(np.array(val,'float32'))
      y_train.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
      X_test.append(np.array(val,'float32'))
      y_test.append(row['emotion'])
  except:
    print("Error occured")

In [9]:
len(X_train)

28709

In [10]:
len(X_test)

3589

In [11]:
X_train = np.array(X_train,'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test, 'float32')

In [12]:
X_train = X_train - np.mean(X_train,axis=0)
X_train = X_train/np.std(X_train,axis=0)

X_test = X_test - np.mean(X_test,axis=0)
X_test = X_test/np.std(X_test,axis=0)

In [13]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

In [14]:
X_train = X_train.reshape(X_train.shape[0],width,height,1)
X_test = X_test.reshape(X_test.shape[0],width,height,1)

In [15]:
y_train = np_utils.to_categorical(y_train, num_classes=num_labels)
y_test = np_utils.to_categorical(y_test, num_classes=num_labels)

In [24]:
model = Sequential()

In [25]:
model.add(Conv2D(4*num_features, (3,3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(4*num_features, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))

In [26]:
model.add(Conv2D(2*num_features, (3,3), activation='relu'))
model.add(Conv2D(2*num_features, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))

In [27]:
model.add(Conv2D(num_features, (3,3), activation='relu'))
model.add(Conv2D(num_features, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(2*2*2*2*num_features, activation='relu'))
model.add(Dropout(0.2))

In [30]:
model.add(Dense(2*2*2*2*num_features, activation='relu'))
model.add(Dropout(0.2))

In [31]:
model.add(Dense(num_labels, activation='softmax'))

In [32]:
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [33]:
model.fit(X_train,y_train,batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(X_test,y_test),shuffle=True)

Epoch 1/100
449/449 [==============================] - 50s 80ms/step - loss: 1.7220 - accuracy: 0.2931 - val_loss: 1.5824 - val_accuracy: 0.3725
Epoch 2/100
449/449 [==============================] - 34s 75ms/step - loss: 1.5167 - accuracy: 0.4012 - val_loss: 1.4226 - val_accuracy: 0.4472
Epoch 3/100
449/449 [==============================] - 33s 74ms/step - loss: 1.4124 - accuracy: 0.4504 - val_loss: 1.3278 - val_accuracy: 0.4792
Epoch 4/100
449/449 [==============================] - 33s 75ms/step - loss: 1.3424 - accuracy: 0.4829 - val_loss: 1.2835 - val_accuracy: 0.4996
Epoch 5/100
449/449 [==============================] - 33s 74ms/step - loss: 1.2955 - accuracy: 0.5014 - val_loss: 1.2653 - val_accuracy: 0.5063
Epoch 6/100
449/449 [==============================] - 33s 74ms/step - loss: 1.2586 - accuracy: 0.5177 - val_loss: 1.2192 - val_accuracy: 0.5383
Epoch 7/100
449/449 [==============================] - 33s 74ms/step - loss: 1.2243 - accuracy: 0.5278 - val_loss: 1.2340 - val_ac

In [37]:
fer_json = model.to_json()
with open("Emotion_Model.json","w") as f:
  f.write(fer_json)
model.save_weights("Emotion_Model.h5")